In [2]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

### Instantiate the video game reviews task with a randomly chosen seed

In [ ]:
seed = np.random.randint(2**32 - 1)

task = VideogameReviewsTask(seed=seed)

### Create a tfdv schema by first auto-infering it from training data and then adjusting it

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

### Train the baseline model for the task

In [4]:
model = task.fit_baseline_model(task.train_data, task.train_labels)

### Run a stress test for the schema and the model with 250 randomly chosen corruptions, and mark performance drops of 3% as failures

In [6]:
stress_test = SchemaStresstest()
results = stress_test.run(task, model, schema, num_corruptions=250, performance_threshold=.03)

BrokenCharacters: {'column': 'title_and_review_text', 'fraction': 0.64}
Scaling: {'column': 'star_rating', 'fraction': 0.51}
MissingValues: {'column': 'title_and_review_text', 'fraction': 0.36, 'na_value': '', 'missingness': 'MNAR'}
GaussianNoise: {'column': 'star_rating', 'fraction': 0.32}
BrokenCharacters: {'column': 'verified_purchase', 'fraction': 0.59}
SwappedValues: {'column_a': 'title_and_review_text', 'column_b': 'product_title', 'fraction': 0.03}
MissingValues: {'column': 'product_title', 'fraction': 0.26, 'na_value': '', 'missingness': 'MAR'}
BrokenCharacters: {'column': 'product_id', 'fraction': 0.37}
MissingValues: {'column': 'product_id', 'fraction': 0.54, 'na_value': '', 'missingness': 'MCAR'}
BrokenCharacters: {'column': 'marketplace', 'fraction': 0.48}
MissingValues: {'column': 'marketplace', 'fraction': 0.05, 'na_value': '', 'missingness': 'MNAR'}
BrokenCharacters: {'column': 'review_body', 'fraction': 0.23}
BrokenCharacters: {'column': 'product_title', 'fraction': 0.3

BrokenCharacters: {'column': 'product_parent', 'fraction': 0.08}
MissingValues: {'column': 'vine', 'fraction': 0.67, 'na_value': '', 'missingness': 'MCAR'}
MissingValues: {'column': 'review_id', 'fraction': 0.08, 'na_value': '', 'missingness': 'MCAR'}
BrokenCharacters: {'column': 'product_id', 'fraction': 0.95}
MissingValues: {'column': 'verified_purchase', 'fraction': 0.6, 'na_value': '', 'missingness': 'MNAR'}
MissingValues: {'column': 'customer_id', 'fraction': 0.07, 'na_value': '', 'missingness': 'MCAR'}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.54}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.34}
MissingValues: {'column': 'customer_id', 'fraction': 0.74, 'na_value': '', 'missingness': 'MAR'}
BrokenCharacters: {'column': 'verified_purchase', 'fraction': 0.07}
MissingValues: {'column': 'product_parent', 'fraction': 0.23, 'na_value': '', 'missingness': 'MAR'}
MissingValues: {'column': 'review_body', 'fraction': 0.9, 'na_value': '', 'missingness': 'MNAR'}


MissingValues: {'column': 'vine', 'fraction': 0.37, 'na_value': '', 'missingness': 'MAR'}
BrokenCharacters: {'column': 'review_body', 'fraction': 0.38}
BrokenCharacters: {'column': 'review_id', 'fraction': 0.42}
MissingValues: {'column': 'review_body', 'fraction': 0.21, 'na_value': '', 'missingness': 'MNAR'}
BrokenCharacters: {'column': 'verified_purchase', 'fraction': 0.66}
BrokenCharacters: {'column': 'product_parent', 'fraction': 0.91}
BrokenCharacters: {'column': 'product_title', 'fraction': 0.76}
BrokenCharacters: {'column': 'verified_purchase', 'fraction': 0.22}
BrokenCharacters: {'column': 'product_parent', 'fraction': 0.15}
MissingValues: {'column': 'product_parent', 'fraction': 0.54, 'na_value': '', 'missingness': 'MNAR'}
SwappedValues: {'column_a': 'vine', 'column_b': 'customer_id', 'fraction': 0.12}
BrokenCharacters: {'column': 'review_body', 'fraction': 0.43}
MissingValues: {'column': 'product_parent', 'fraction': 0.79, 'na_value': '', 'missingness': 'MCAR'}
GaussianNoise: 

### Look at the dataframe containing the results

In [7]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,BrokenCharacters: {'column': 'title_and_review...,FN,{},0.793712,0.736502
1,"Scaling: {'column': 'star_rating', 'fraction':...",FN,{},0.793712,0.562934
2,MissingValues: {'column': 'title_and_review_te...,FN,{},0.793712,0.752936
3,"GaussianNoise: {'column': 'star_rating', 'frac...",TP,"{'star_rating': description: """"\nseverity: ERR...",0.793712,0.756830
4,BrokenCharacters: {'column': 'verified_purchas...,TN,{},0.793712,0.793712
...,...,...,...,...,...
245,BrokenCharacters: {'column': 'verified_purchas...,TN,{},0.793712,0.793712
246,"BrokenCharacters: {'column': 'marketplace', 'f...",FP,"{'marketplace': description: ""Examples contain...",0.793712,0.793712
247,"MissingValues: {'column': 'verified_purchase',...",FP,"{'verified_purchase': description: ""Examples c...",0.793712,0.774270
248,"BrokenCharacters: {'column': 'vine', 'fraction...",TN,{},0.793712,0.793712
